In [1]:
# should already be activated, but just a reminder where ya are :)  
# !conda activate wool_sucking_nn

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats 

%matplotlib inline

In [2]:
# import data
excel_file = pd.read_excel('..\Data\cleaned_and_split_data.xlsx', sheet_name=None)

# Unpack the dataframes into separate variables
X_train = excel_file['X_train']
X_test = excel_file['X_test']
y_train = excel_file['y_train']
y_test = excel_file['y_test']

In [4]:
print(X_train.shape)
X_train.head()

(3965, 7)


,Neuter_status,Breed_group,Aggression_owner,Aggression_cats,Shyness_novel,Shyness_strangers,Grooming
0,1,SIB,1,2,4,4,3
1,1,HCS,1,2,4,4,2
2,0,ORI,1,1,1,1,1
3,1,MCO,1,1,2,2,3
4,1,HCS,1,1,2,1,2


In [5]:
print(X_test.shape)
X_test.head()

(1700, 7)


,Neuter_status,Breed_group,Aggression_owner,Aggression_cats,Shyness_novel,Shyness_strangers,Grooming
0,1,SIB,1,3,3,1,1
1,1,MCO,1,1,2,1,1
2,1,RUS,1,1,4,4,1
3,0,EUR,2,2,2,2,1
4,1,NFO,1,2,3,2,1


In [6]:
print(y_train.shape)
y_train.head()

(3965, 1)


,Wool_sucking_binary
0,0
1,0
2,0
3,0
4,0


In [7]:
print(y_test.shape)
y_test.head()

(1700, 1)


,Wool_sucking_binary
0,1
1,1
2,1
3,0
4,1


## To record model metrics

In [3]:
# dataframe for scores amongst models
master_scores = pd.DataFrame(columns=['Model','Recall', 'F1', 'Precision', 'Accuracy'])

In [4]:
# add_to_master(model_description, y_true, y_predicted, binary=True)
def add_to_master(model_description, y_test, y_pred, binary=True):
    '''
    Adds a new row to the running score DataFrame `master_scores` 
    and functions for adding both binary and multiclass scores. 
    '''
    global master_scores
    if binary == True:
        # Evalution Metrics
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        # add to master_scores
        new_row = {
            'Model': model_description,
            'Recall': round(recall,3), 
            'F1': round(f1, 3), 
            'Precision': round(precision, 3), 
            'Accuracy': round(accuracy, 3)}
        
        master_scores = master_scores.append(new_row, ignore_index=True)   
    
    else:
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='macro') # I chose macro since we have less of the sucklers 
        recall = recall_score(y_test, y_pred, average='macro')       # and I want their accuracy to be meaningful
        f1 = f1_score(y_test, y_pred, average='macro')


        # add to master_scores
        new_row = {
            'Model': model_description,
            'Recall': round(recall,3), 
            'F1': round(f1, 3), 
            'Precision': round(precision, 3), 
            'Accuracy': round(accuracy, 3)}
        master_scores = master_scores.append(new_row, ignore_index=True)
    return master_scores

# Prep for tensorflow.keras

In [5]:
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import accuracy_score,precision_score, recall_score, f1_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
# for reproducibility 
tf.random.set_seed(42)
np.random.seed(42)

In [6]:
# one-hot breed 
X_train_encoded = pd.get_dummies(X_train, columns=['Breed_group'])
X_test_encoded = pd.get_dummies(X_test, columns=['Breed_group'])

In [8]:
X_train_encoded.shape

(3965, 25)

## Plain Jane

In [9]:
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(len(X_train_encoded.columns),)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

In [10]:
def recall(y_true, y_pred):
    true_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
    possible_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + tf.keras.backend.epsilon())
    return recall

tf.keras.utils.get_custom_objects()['recall'] = recall

In [11]:
model.compile(optimizer='adam', 
             loss='binary_crossentropy',
             metrics=['accuracy', 'recall'])

In [12]:
model.fit(X_train_encoded, y_train, epochs=10, batch_size=32)

Epoch 1/10
124/124 [==============================] - 1s 932us/step - loss: 0.6265 - accuracy: 0.6815 - recall: 0.0133
Epoch 2/10
124/124 [==============================] - 0s 941us/step - loss: 0.6032 - accuracy: 0.6870 - recall: 0.1315
Epoch 3/10
124/124 [==============================] - 0s 932us/step - loss: 0.5939 - accuracy: 0.6921 - recall: 0.1803
Epoch 4/10
124/124 [==============================] - 0s 965us/step - loss: 0.5902 - accuracy: 0.6941 - recall: 0.1556
Epoch 5/10
124/124 [==============================] - 0s 949us/step - loss: 0.5852 - accuracy: 0.6951 - recall: 0.1767
Epoch 6/10
124/124 [==============================] - 0s 945us/step - loss: 0.5843 - accuracy: 0.6941 - recall: 0.2047
Epoch 7/10
124/124 [==============================] - 0s 957us/step - loss: 0.5823 - accuracy: 0.6936 - recall: 0.2127
Epoch 8/10
124/124 [==============================] - 0s 957us/step - loss: 0.5775 - accuracy: 0.6981 - recall: 0.2054
Epoch 9/10
124/124 [============================

In [13]:
test_loss, test_acc, test_recall = model.evaluate(X_test_encoded, y_test)
print('Test Accuracy:', test_acc)
print('Test Loss:', test_loss)
print('Test Recall:', test_recall)

54/54 [==============================] - 0s 734us/step - loss: 0.5869 - accuracy: 0.6824 - recall: 0.2525
Test Accuracy: 0.6823529601097107
Test Loss: 0.5868842601776123
Test Recall: 0.2525482475757599


In [14]:
y_pred = model.predict(X_test_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
# add_to_master(model_description, y_true, y_predicted, binary=True)
add_to_master("1 hidden, 64 nodes per, adam", y_test, y_pred)
cm

54/54 [==============================] - 0s 659us/step


C:\Users\gwent\AppData\Local\Temp\ipykernel_19588\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[1019,  140],
       [ 400,  141]], dtype=int64)

In [15]:
master_scores

,Model,Recall,F1,Precision,Accuracy
0,"1 hidden, 64 nodes per, adam",0.261,0.343,0.502,0.682


Not doing too hot, but also seems to be doing better than initial models in Cat-pstone 1. 

## Adagrad optimizer

In [16]:
model2 = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(len(X_train_encoded.columns),)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

In [17]:
# Adagrad better for when feataures vary in magnitude and frequency 
optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.01)
model2.compile(optimizer=optimizer, 
             loss='binary_crossentropy',
             metrics=['accuracy', 'recall'])

In [18]:
model2.fit(X_train_encoded, y_train, epochs=10, batch_size=32)

Epoch 1/10
124/124 [==============================] - 0s 957us/step - loss: 0.6313 - accuracy: 0.6787 - recall: 0.0244
Epoch 2/10
124/124 [==============================] - 0s 916us/step - loss: 0.6187 - accuracy: 0.6827 - recall: 0.0158
Epoch 3/10
124/124 [==============================] - 0s 973us/step - loss: 0.6118 - accuracy: 0.6842 - recall: 0.0573
Epoch 4/10
124/124 [==============================] - 0s 957us/step - loss: 0.6075 - accuracy: 0.6845 - recall: 0.0582
Epoch 5/10
124/124 [==============================] - 0s 908us/step - loss: 0.6040 - accuracy: 0.6860 - recall: 0.0803
Epoch 6/10
124/124 [==============================] - 0s 900us/step - loss: 0.6007 - accuracy: 0.6873 - recall: 0.1213
Epoch 7/10
124/124 [==============================] - 0s 859us/step - loss: 0.5986 - accuracy: 0.6908 - recall: 0.1207
Epoch 8/10
124/124 [==============================] - 0s 868us/step - loss: 0.5969 - accuracy: 0.6878 - recall: 0.1275
Epoch 9/10
124/124 [============================

In [19]:
test_loss, test_acc, test_recall = model2.evaluate(X_test_encoded, y_test)
print('Test Accuracy:', test_acc)
print('Test Loss:', test_loss)
print('Test Recall:', test_recall)

54/54 [==============================] - 0s 753us/step - loss: 0.5976 - accuracy: 0.6818 - recall: 0.1763
Test Accuracy: 0.6817647218704224
Test Loss: 0.5975832939147949
Test Recall: 0.1762544810771942


In [20]:
y_pred = model2.predict(X_test_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("1 hidden, 64 nodes per, adagrad", y_test, y_pred)
cm

54/54 [==============================] - 0s 640us/step


C:\Users\gwent\AppData\Local\Temp\ipykernel_19588\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[1061,   98],
       [ 443,   98]], dtype=int64)

## Scaled with Plain Jane and Adagrad

In [21]:
scaler = StandardScaler()
scaler.fit(X_train_encoded)

StandardScaler()

In [22]:
X_train_scaled_encoded = scaler.transform(X_train_encoded)
X_test_scaled_encoded = scaler.transform(X_test_encoded)

In [23]:
X_train_scaled_encoded.shape

(3965, 25)

In [24]:
# Plain Jane + scaled
model3 = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(25,)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model3.compile(optimizer='adam', 
             loss='binary_crossentropy',
             metrics=['accuracy', 'recall'])

model3.fit(X_train_scaled_encoded, y_train, epochs=10, batch_size=32)

test_loss, test_acc, test_recall = model3.evaluate(X_test_scaled_encoded, y_test)
print('Test Accuracy:', test_acc)
print('Test Loss:', test_loss)
print('Test Recall:', test_recall)

y_pred = model3.predict(X_test_scaled_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("1 hidden, 64 nodes per, adam, scaled", y_test, y_pred)
cm

Epoch 1/10
124/124 [==============================] - 0s 900us/step - loss: 0.6084 - accuracy: 0.6782 - recall: 0.1150
Epoch 2/10
124/124 [==============================] - 0s 925us/step - loss: 0.5878 - accuracy: 0.6910 - recall: 0.1646
Epoch 3/10
124/124 [==============================] - 0s 932us/step - loss: 0.5825 - accuracy: 0.6984 - recall: 0.2363
Epoch 4/10
124/124 [==============================] - 0s 941us/step - loss: 0.5791 - accuracy: 0.7026 - recall: 0.1833
Epoch 5/10
124/124 [==============================] - 0s 924us/step - loss: 0.5742 - accuracy: 0.7029 - recall: 0.2126
Epoch 6/10
124/124 [==============================] - 0s 957us/step - loss: 0.5732 - accuracy: 0.7011 - recall: 0.2244
Epoch 7/10
124/124 [==============================] - 0s 941us/step - loss: 0.5715 - accuracy: 0.7026 - recall: 0.2265
Epoch 8/10
124/124 [==============================] - 0s 932us/step - loss: 0.5676 - accuracy: 0.7142 - recall: 0.2564
Epoch 9/10
124/124 [============================

C:\Users\gwent\AppData\Local\Temp\ipykernel_19588\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[1021,  138],
       [ 395,  146]], dtype=int64)

In [25]:
# adagrad + scaled 
model4 = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(25,)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# Adagrad better for when feataures vary in magnitude and frequency 
optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.01)
model4.compile(optimizer=optimizer, 
             loss='binary_crossentropy',
             metrics=['accuracy', 'recall'])

model4.fit(X_train_scaled_encoded, y_train, epochs=10, batch_size=32)

test_loss, test_acc, test_recall = model4.evaluate(X_test_scaled_encoded, y_test)
print('Test Accuracy:', test_acc)
print('Test Loss:', test_loss)
print('Test Recall:', test_recall)

y_pred = model4.predict(X_test_scaled_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("1 hidden, 64 nodes per, adagrad, scaled", y_test, y_pred)
cm

Epoch 1/10
124/124 [==============================] - 0s 851us/step - loss: 0.6276 - accuracy: 0.6734 - recall: 0.0936
Epoch 2/10
124/124 [==============================] - 0s 868us/step - loss: 0.6014 - accuracy: 0.6873 - recall: 0.1033
Epoch 3/10
124/124 [==============================] - 0s 868us/step - loss: 0.5932 - accuracy: 0.6923 - recall: 0.1547
Epoch 4/10
124/124 [==============================] - 0s 892us/step - loss: 0.5888 - accuracy: 0.6905 - recall: 0.1493
Epoch 5/10
124/124 [==============================] - 0s 859us/step - loss: 0.5859 - accuracy: 0.6961 - recall: 0.1655
Epoch 6/10
124/124 [==============================] - 0s 876us/step - loss: 0.5837 - accuracy: 0.6981 - recall: 0.1805
Epoch 7/10
124/124 [==============================] - 0s 876us/step - loss: 0.5819 - accuracy: 0.6968 - recall: 0.1753
Epoch 8/10
124/124 [==============================] - 0s 876us/step - loss: 0.5806 - accuracy: 0.7009 - recall: 0.1982
Epoch 9/10
124/124 [============================

C:\Users\gwent\AppData\Local\Temp\ipykernel_19588\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[1054,  105],
       [ 447,   94]], dtype=int64)

In [26]:
master_scores.sort_values('Recall', ascending=False)

,Model,Recall,F1,Precision,Accuracy
2,"1 hidden, 64 nodes per, adam, scaled",0.270,0.354,0.514,0.686
0,"1 hidden, 64 nodes per, adam",0.261,0.343,0.502,0.682
1,"1 hidden, 64 nodes per, adagrad",0.181,0.266,0.500,0.682
3,"1 hidden, 64 nodes per, adagrad, scaled",0.174,0.254,0.472,0.675


In [27]:
def create_mlp(nodes, optimizer, X_train):
    # Define the model architecture
    model = keras.Sequential([
    keras.layers.Dense(nodes, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(nodes, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

    model.compile(optimizer=optimizer, 
             loss='binary_crossentropy',
             metrics=['accuracy', 'recall'])
    
    return model

In [28]:
model5 = create_mlp(100, 'adam', X_train_scaled_encoded)

In [30]:
model5.fit(X_train_scaled_encoded, y_train, epochs=10, batch_size=32)

test_loss, test_acc, test_recall = model5.evaluate(X_test_scaled_encoded, y_test)
print('Test Accuracy:', test_acc)
print('Test Loss:', test_loss)
print('Test Recall:', test_recall)

y_pred = model5.predict(X_test_scaled_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("1 hidden, 100 nodes, adam", y_test, y_pred)
cm

Epoch 1/10
124/124 [==============================] - 1s 1ms/step - loss: 0.6141 - accuracy: 0.6709 - recall: 0.0954
Epoch 2/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5865 - accuracy: 0.6963 - recall: 0.1866
Epoch 3/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5801 - accuracy: 0.6976 - recall: 0.2339
Epoch 4/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5754 - accuracy: 0.7019 - recall: 0.1976
Epoch 5/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5699 - accuracy: 0.7067 - recall: 0.2429
Epoch 6/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5699 - accuracy: 0.7110 - recall: 0.2607
Epoch 7/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5672 - accuracy: 0.7082 - recall: 0.2752
Epoch 8/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5631 - accuracy: 0.7163 - recall: 0.2719
Epoch 9/10
124/124 [==============================] - 0s 1ms/ste

C:\Users\gwent\AppData\Local\Temp\ipykernel_19588\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[984, 175],
       [393, 148]], dtype=int64)

In [31]:
model6 = create_mlp(32, 'adam', X_train_scaled_encoded)
model6.fit(X_train_scaled_encoded, y_train, epochs=10, batch_size=32)

test_loss, test_acc, test_recall = model6.evaluate(X_test_scaled_encoded, y_test)
print('Test Accuracy:', test_acc)
print('Test Loss:', test_loss)
print('Test Recall:', test_recall)

y_pred = model6.predict(X_test_scaled_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("1 hidden, 32 nodes, adam", y_test, y_pred)
cm

Epoch 1/10
124/124 [==============================] - 0s 989us/step - loss: 0.6270 - accuracy: 0.6593 - recall: 0.0858
Epoch 2/10
124/124 [==============================] - 0s 908us/step - loss: 0.5947 - accuracy: 0.6880 - recall: 0.0846
Epoch 3/10
124/124 [==============================] - 0s 908us/step - loss: 0.5894 - accuracy: 0.6943 - recall: 0.1676
Epoch 4/10
124/124 [==============================] - 0s 892us/step - loss: 0.5842 - accuracy: 0.6933 - recall: 0.1562
Epoch 5/10
124/124 [==============================] - 0s 923us/step - loss: 0.5800 - accuracy: 0.7006 - recall: 0.1970
Epoch 6/10
124/124 [==============================] - 0s 932us/step - loss: 0.5778 - accuracy: 0.6974 - recall: 0.2139
Epoch 7/10
124/124 [==============================] - 0s 884us/step - loss: 0.5759 - accuracy: 0.6986 - recall: 0.2190
Epoch 8/10
124/124 [==============================] - 0s 900us/step - loss: 0.5732 - accuracy: 0.7011 - recall: 0.2235
Epoch 9/10
124/124 [============================

C:\Users\gwent\AppData\Local\Temp\ipykernel_19588\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[1019,  140],
       [ 415,  126]], dtype=int64)

In [32]:
model7 = create_mlp(250, 'adam', X_train_scaled_encoded)
model7.fit(X_train_scaled_encoded, y_train, epochs=10, batch_size=32)

test_loss, test_acc, test_recall = model7.evaluate(X_test_scaled_encoded, y_test)
print('Test Accuracy:', test_acc)
print('Test Loss:', test_loss)
print('Test Recall:', test_recall)

y_pred = model7.predict(X_test_scaled_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("1 hidden, 250 nodes, adam", y_test, y_pred)
cm

Epoch 1/10
124/124 [==============================] - 1s 1ms/step - loss: 0.6082 - accuracy: 0.6827 - recall: 0.1674
Epoch 2/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5884 - accuracy: 0.6958 - recall: 0.2142
Epoch 3/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5803 - accuracy: 0.6991 - recall: 0.2312
Epoch 4/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5783 - accuracy: 0.7016 - recall: 0.2066
Epoch 5/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5693 - accuracy: 0.7100 - recall: 0.2474
Epoch 6/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5703 - accuracy: 0.7082 - recall: 0.2598
Epoch 7/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5672 - accuracy: 0.7084 - recall: 0.2736
Epoch 8/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5609 - accuracy: 0.7137 - recall: 0.2802
Epoch 9/10
124/124 [==============================] - 0s 1ms/ste

C:\Users\gwent\AppData\Local\Temp\ipykernel_19588\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[986, 173],
       [380, 161]], dtype=int64)

In [33]:
# definitely overfits
model8 = create_mlp(300, 'adam', X_train_scaled_encoded)
model8.fit(X_train_scaled_encoded, y_train, epochs=10, batch_size=32)

test_loss, test_acc, test_recall = model8.evaluate(X_test_scaled_encoded, y_test)
print('Test Accuracy:', test_acc)
print('Test Loss:', test_loss)
print('Test Recall:', test_recall)

y_pred = model8.predict(X_test_scaled_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("1 hidden, 300 nodes, adam", y_test, y_pred)
cm

Epoch 1/10
124/124 [==============================] - 0s 1ms/step - loss: 0.6090 - accuracy: 0.6817 - recall: 0.1648
Epoch 2/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5882 - accuracy: 0.6963 - recall: 0.2078
Epoch 3/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5797 - accuracy: 0.6968 - recall: 0.2331
Epoch 4/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5777 - accuracy: 0.6994 - recall: 0.2063
Epoch 5/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5691 - accuracy: 0.7105 - recall: 0.2526
Epoch 6/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5713 - accuracy: 0.7042 - recall: 0.2523
Epoch 7/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5680 - accuracy: 0.7082 - recall: 0.2702
Epoch 8/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5621 - accuracy: 0.7122 - recall: 0.2652
Epoch 9/10
124/124 [==============================] - 0s 1ms/ste

C:\Users\gwent\AppData\Local\Temp\ipykernel_19588\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[1004,  155],
       [ 391,  150]], dtype=int64)

In [34]:
master_scores.sort_values('Recall', ascending=False)

,Model,Recall,F1,Precision,Accuracy
6,"1 hidden, 250 nodes, adam",0.298,0.368,0.482,0.675
7,"1 hidden, 300 nodes, adam",0.277,0.355,0.492,0.679
4,"1 hidden, 100 nodes, adam",0.274,0.343,0.458,0.666
2,"1 hidden, 64 nodes per, adam, scaled",0.270,0.354,0.514,0.686
0,"1 hidden, 64 nodes per, adam",0.261,0.343,0.502,0.682
5,"1 hidden, 32 nodes, adam",0.233,0.312,0.474,0.674
1,"1 hidden, 64 nodes per, adagrad",0.181,0.266,0.500,0.682
3,"1 hidden, 64 nodes per, adagrad, scaled",0.174,0.254,0.472,0.675


Okay so we are pretty similar in performance to the logistic regression models from cat-pstone 1 on every metric except recall (and the part it plays in F1, of course). Let's see if over or undersampling with the top performing model architecture (model1). 

In [35]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

In [36]:
X_train_scaled_encoded.shape

(3965, 25)

In [37]:
rus = RandomUnderSampler()
X_resampled_rus, y_resampled_rus = rus.fit_resample(X_train_scaled_encoded, y_train)

ros = RandomOverSampler()
X_resampled_ros, y_resampled_ros = ros.fit_resample(X_train_scaled_encoded, y_train)

In [38]:
# RUS 
model9 = create_mlp(64, 'adam', X_train_scaled_encoded)
model9.fit(X_resampled_rus, y_resampled_rus, epochs=10, batch_size=32)

y_pred = model9.predict(X_test_scaled_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("RUS 1 hidden, 64 nodes, adam", y_test, y_pred)
cm

Epoch 1/10
79/79 [==============================] - 0s 908us/step - loss: 0.6752 - accuracy: 0.5860 - recall: 0.5806
Epoch 2/10
79/79 [==============================] - 0s 908us/step - loss: 0.6497 - accuracy: 0.6228 - recall: 0.6084
Epoch 3/10
79/79 [==============================] - 0s 921us/step - loss: 0.6425 - accuracy: 0.6343 - recall: 0.6528
Epoch 4/10
79/79 [==============================] - 0s 921us/step - loss: 0.6359 - accuracy: 0.6371 - recall: 0.6382
Epoch 5/10
79/79 [==============================] - 0s 921us/step - loss: 0.6322 - accuracy: 0.6438 - recall: 0.6356
Epoch 6/10
79/79 [==============================] - 0s 921us/step - loss: 0.6284 - accuracy: 0.6545 - recall: 0.6800
Epoch 7/10
79/79 [==============================] - 0s 933us/step - loss: 0.6232 - accuracy: 0.6529 - recall: 0.6482
Epoch 8/10
79/79 [==============================] - 0s 946us/step - loss: 0.6188 - accuracy: 0.6636 - recall: 0.6866
Epoch 9/10
79/79 [==============================] - 0s 1ms/step 

C:\Users\gwent\AppData\Local\Temp\ipykernel_19588\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[722, 437],
       [217, 324]], dtype=int64)

In [39]:
# ROS 
model10 = create_mlp(64, 'adam', X_train_scaled_encoded)
model10.fit(X_resampled_ros, y_resampled_ros, epochs=10, batch_size=32)

y_pred = model10.predict(X_test_scaled_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("ROS 1 hidden, 64 nodes, adam", y_test, y_pred)
cm

Epoch 1/10
169/169 [==============================] - 0s 885us/step - loss: 0.6656 - accuracy: 0.5982 - recall: 0.5904
Epoch 2/10
169/169 [==============================] - 0s 908us/step - loss: 0.6437 - accuracy: 0.6343 - recall: 0.6210
Epoch 3/10
169/169 [==============================] - 0s 926us/step - loss: 0.6367 - accuracy: 0.6411 - recall: 0.6393
Epoch 4/10
169/169 [==============================] - 0s 938us/step - loss: 0.6302 - accuracy: 0.6482 - recall: 0.6408
Epoch 5/10
169/169 [==============================] - 0s 914us/step - loss: 0.6214 - accuracy: 0.6539 - recall: 0.6399
Epoch 6/10
169/169 [==============================] - 0s 968us/step - loss: 0.6183 - accuracy: 0.6583 - recall: 0.6615
Epoch 7/10
169/169 [==============================] - 0s 1ms/step - loss: 0.6141 - accuracy: 0.6539 - recall: 0.6499
Epoch 8/10
169/169 [==============================] - 0s 944us/step - loss: 0.6099 - accuracy: 0.6643 - recall: 0.6497
Epoch 9/10
169/169 [==============================

C:\Users\gwent\AppData\Local\Temp\ipykernel_19588\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[785, 374],
       [258, 283]], dtype=int64)

In [40]:
# RUS 
model11 = create_mlp(64, 'adam', X_train_scaled_encoded)
model11.fit(X_resampled_rus, y_resampled_rus, epochs=10, batch_size=25)

y_pred = model11.predict(X_test_scaled_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("RUS, adam, batch 25", y_test, y_pred)
cm

Epoch 1/10
101/101 [==============================] - 0s 947us/step - loss: 0.6770 - accuracy: 0.5899 - recall: 0.5762
Epoch 2/10
101/101 [==============================] - 0s 967us/step - loss: 0.6512 - accuracy: 0.6244 - recall: 0.6237
Epoch 3/10
101/101 [==============================] - 0s 928us/step - loss: 0.6442 - accuracy: 0.6331 - recall: 0.6398
Epoch 4/10
101/101 [==============================] - 0s 918us/step - loss: 0.6375 - accuracy: 0.6426 - recall: 0.6361
Epoch 5/10
101/101 [==============================] - 0s 908us/step - loss: 0.6335 - accuracy: 0.6506 - recall: 0.6458
Epoch 6/10
101/101 [==============================] - 0s 928us/step - loss: 0.6271 - accuracy: 0.6494 - recall: 0.6464
Epoch 7/10
101/101 [==============================] - 0s 937us/step - loss: 0.6241 - accuracy: 0.6565 - recall: 0.6579
Epoch 8/10
101/101 [==============================] - 0s 918us/step - loss: 0.6192 - accuracy: 0.6609 - recall: 0.6550
Epoch 9/10
101/101 [============================

C:\Users\gwent\AppData\Local\Temp\ipykernel_19588\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[742, 417],
       [240, 301]], dtype=int64)

In [42]:
y_train.shape

(3965, 1)

## out of curiosity, does MLP still predict closely to logistic regression with grooming dropped?

In [44]:
X_train_no_groom = X_train.drop(columns=['Grooming'])
X_test_no_groom = X_test.drop(columns=['Grooming'])

print(X_train_no_groom.shape)
print(X_test_no_groom.shape)

# one-hot breed 
X_train_encoded_no_groom = pd.get_dummies(X_train_no_groom, columns=['Breed_group'])
X_test_encoded_no_groom = pd.get_dummies(X_test_no_groom, columns=['Breed_group'])

print(X_train_encoded_no_groom.shape)
print(X_test_encoded_no_groom.shape)

scaler = StandardScaler()
scaler.fit(X_train_encoded)
X_train_scaled_encoded_ng = scaler.transform(X_train_encoded)
X_test_scaled_encoded_ng = scaler.transform(X_test_encoded)

print(X_train_scaled_encoded_ng.shape)
print(X_test_scaled_encoded_ng.shape)

rus = RandomUnderSampler()
X_resampled_rus_ng, y_resampled_rus_ng = rus.fit_resample(X_train_scaled_encoded, y_train)

(3965, 6)
(1700, 6)
(3965, 24)
(1700, 24)
(3965, 25)
(1700, 25)


In [45]:
print(X_train_scaled_encoded_ng.shape)
print(X_resampled_rus_ng.shape)
print(X_train_scaled_encoded.shape)

(3965, 25)
(2524, 25)
(3965, 25)


In [46]:
# RUS 64 adam scaled no grooming
model11 = create_mlp(64, 'adam', X_resampled_rus_ng)
model11.fit(X_resampled_rus_ng, y_resampled_rus_ng, epochs=10, batch_size=32)

y_pred = model11.predict(X_test_scaled_encoded_ng)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("no groom, RUS, 64, adam", y_test, y_pred)
cm

Epoch 1/10
79/79 [==============================] - 0s 895us/step - loss: 0.6794 - accuracy: 0.5836 - recall: 0.6108
Epoch 2/10
79/79 [==============================] - 0s 946us/step - loss: 0.6560 - accuracy: 0.6090 - recall: 0.6237
Epoch 3/10
79/79 [==============================] - 0s 933us/step - loss: 0.6499 - accuracy: 0.6268 - recall: 0.6349
Epoch 4/10
79/79 [==============================] - 0s 921us/step - loss: 0.6457 - accuracy: 0.6288 - recall: 0.6319
Epoch 5/10
79/79 [==============================] - 0s 933us/step - loss: 0.6410 - accuracy: 0.6367 - recall: 0.6397
Epoch 6/10
79/79 [==============================] - 0s 933us/step - loss: 0.6375 - accuracy: 0.6383 - recall: 0.6510
Epoch 7/10
79/79 [==============================] - 0s 921us/step - loss: 0.6335 - accuracy: 0.6359 - recall: 0.6273
Epoch 8/10
79/79 [==============================] - 0s 933us/step - loss: 0.6298 - accuracy: 0.6426 - recall: 0.6493
Epoch 9/10
79/79 [==============================] - 0s 972us/ste

C:\Users\gwent\AppData\Local\Temp\ipykernel_19588\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[747, 412],
       [216, 325]], dtype=int64)

In [47]:
master_scores.sort_values('Recall', ascending=False).head()

,Model,Recall,F1,Precision,Accuracy
11,"no groom, RUS, 64, adam",0.601,0.509,0.441,0.631
8,"RUS 1 hidden, 64 nodes, adam",0.599,0.498,0.426,0.615
10,"RUS, adam, batch 25",0.556,0.478,0.419,0.614
9,"ROS 1 hidden, 64 nodes, adam",0.523,0.472,0.431,0.628
6,"1 hidden, 250 nodes, adam",0.298,0.368,0.482,0.675


In [48]:
import time
# training time 
start = time.time()
model10.fit(X_resampled_ros, y_resampled_ros, epochs=10, batch_size=32)
end = time.time()
training_time = end - start

p_start = time.time()
y_pred = model10.predict(X_test_encoded)
p_end = time.time()
predict_time = p_end - p_start

Epoch 1/10
169/169 [==============================] - 0s 926us/step - loss: 0.5968 - accuracy: 0.6726 - recall: 0.6762
Epoch 2/10
169/169 [==============================] - 0s 932us/step - loss: 0.5929 - accuracy: 0.6765 - recall: 0.6864
Epoch 3/10
169/169 [==============================] - 0s 920us/step - loss: 0.5912 - accuracy: 0.6829 - recall: 0.6990
Epoch 4/10
169/169 [==============================] - 0s 932us/step - loss: 0.5881 - accuracy: 0.6811 - recall: 0.6870
Epoch 5/10
169/169 [==============================] - 0s 920us/step - loss: 0.5797 - accuracy: 0.6926 - recall: 0.7016
Epoch 6/10
169/169 [==============================] - 0s 938us/step - loss: 0.5791 - accuracy: 0.6907 - recall: 0.7088
Epoch 7/10
169/169 [==============================] - 0s 938us/step - loss: 0.5758 - accuracy: 0.6894 - recall: 0.6979
Epoch 8/10
169/169 [==============================] - 0s 1ms/step - loss: 0.5727 - accuracy: 0.6948 - recall: 0.7014
Epoch 9/10
169/169 [==============================

# Comparing to a Logistic Regression
In Cat-pstone 1 I had did not prune down the columns nearly as much. I want to see how it does with this new training set. 

In [49]:
from sklearn.linear_model import LogisticRegression

# fit model, + training time 
t_start = time.time()
log_balanced = LogisticRegression(class_weight='balanced').fit(X_train_encoded, (y_train.values.reshape(-1)))
t_end = time.time()
log_training = t_end - t_start

# predict w model, + prediction time 
p_start = time.time()
y_pred = log_balanced.predict(X_test_encoded)
p_end = time.time()
log_predict = p_end - p_start 

# add_to_master(model_description, y_true, y_predicted, binary=True)
add_to_master("class_weight='balanced', LogReg", y_test, y_pred)

C:\Users\gwent\AppData\Local\Temp\ipykernel_19588\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


,Model,Recall,F1,Precision,Accuracy
0,"1 hidden, 64 nodes per, adam",0.261,0.343,0.502,0.682
1,"1 hidden, 64 nodes per, adagrad",0.181,0.266,0.500,0.682
2,"1 hidden, 64 nodes per, adam, scaled",0.270,0.354,0.514,0.686
3,"1 hidden, 64 nodes per, adagrad, scaled",0.174,0.254,0.472,0.675
4,"1 hidden, 100 nodes, adam",0.274,0.343,0.458,0.666
5,"1 hidden, 32 nodes, adam",0.233,0.312,0.474,0.674
6,"1 hidden, 250 nodes, adam",0.298,0.368,0.482,0.675
7,"1 hidden, 300 nodes, adam",0.277,0.355,0.492,0.679
8,"RUS 1 hidden, 64 nodes, adam",0.599,0.498,0.426,0.615
9,"ROS 1 hidden, 64 nodes, adam",0.523,0.472,0.431,0.628


In [50]:
# scaled and logistic regression
log_balanced_scaled = LogisticRegression(class_weight='balanced').fit(X_train_encoded, (y_train.values.reshape(-1)))
y_pred = log_balanced_scaled.predict(X_test_scaled_encoded)

# add_to_master(model_description, y_true, y_predicted, binary=True)
add_to_master("class_weight='balanced', LogReg, scaled", y_test, y_pred);

C:\Users\gwent\anaconda3\envs\wool_sucking_nn\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
C:\Users\gwent\AppData\Local\Temp\ipykernel_19588\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


In [51]:
# no grooming logistic regression 
log_no_groom = LogisticRegression(class_weight='balanced').fit(X_train_scaled_encoded_ng, y_train)
y_pred = log_no_groom.predict(X_test_scaled_encoded_ng)

add_to_master('log reg no grooming', y_test, y_pred)

C:\Users\gwent\anaconda3\envs\wool_sucking_nn\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\gwent\AppData\Local\Temp\ipykernel_19588\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


,Model,Recall,F1,Precision,Accuracy
0,"1 hidden, 64 nodes per, adam",0.261,0.343,0.502,0.682
1,"1 hidden, 64 nodes per, adagrad",0.181,0.266,0.500,0.682
2,"1 hidden, 64 nodes per, adam, scaled",0.270,0.354,0.514,0.686
3,"1 hidden, 64 nodes per, adagrad, scaled",0.174,0.254,0.472,0.675
4,"1 hidden, 100 nodes, adam",0.274,0.343,0.458,0.666
5,"1 hidden, 32 nodes, adam",0.233,0.312,0.474,0.674
6,"1 hidden, 250 nodes, adam",0.298,0.368,0.482,0.675
7,"1 hidden, 300 nodes, adam",0.277,0.355,0.492,0.679
8,"RUS 1 hidden, 64 nodes, adam",0.599,0.498,0.426,0.615
9,"ROS 1 hidden, 64 nodes, adam",0.523,0.472,0.431,0.628


In [52]:
print(f"MLP training time: {training_time} \nMLP predict time: {predict_time} \nLogistic Regression training time: {log_training} \nLogistic Regression predict time: {log_predict}")
train_ratio = round(training_time / log_training, 1)
predict_ratio = round(predict_time / log_predict, 1)
print(f"MLP is {train_ratio} times slower at training than logistic regression. \nMLP is {predict_ratio} times slower at predicting than logistic regression." )
# AND I didn't even include the time spent doing a random undersampling for the MLP, which logistic reg didn't need 

MLP training time: 1.6904628276824951 
MLP predict time: 0.14970064163208008 
Logistic Regression training time: 0.020906686782836914 
Logistic Regression predict time: 0.0009975433349609375
MLP is 80.9 times slower at training than logistic regression. 
MLP is 150.1 times slower at predicting than logistic regression.


In [53]:
# for reference, cat-pstone 1 model was 0.622, 0.519, 0.446, 0.635
master_scores.sort_values('Recall', ascending=False).head()

,Model,Recall,F1,Precision,Accuracy
11,"no groom, RUS, 64, adam",0.601,0.509,0.441,0.631
8,"RUS 1 hidden, 64 nodes, adam",0.599,0.498,0.426,0.615
14,log reg no grooming,0.593,0.504,0.437,0.628
12,"class_weight='balanced', LogReg",0.588,0.502,0.438,0.629
10,"RUS, adam, batch 25",0.556,0.478,0.419,0.614


Given that this model does not perform as well as the cat-pstone 1 model (marginally), I want to see how my best performing model architecture does on the dataset I used with cat-pstone 1 (with a lot more columns!). 

In [54]:
from sklearn.model_selection import train_test_split
df = pd.read_csv('..\Data\cat_clean.csv')

In [55]:
# binarize wool-sucking
df_bin = df.copy()
df_bin['ws_binary'] = df_bin['Wool_sucking'].replace([1, 2, 3, 4, 5, 6, 7], [1, 1, 1, 1, 1, 1, 1])
df_bin.drop(columns='Wool_sucking', inplace=True)

# Recode Behaviour_problem to binary
df_bin['Behaviour_problem'].replace([1, 2, 3], [0, 1, 1], inplace=True)

# encode breed group
df_bin_encoded = pd.get_dummies(df_bin, columns=['Breed_group'])

In [56]:
X = df_bin_encoded.drop(columns=['ws_binary'])
y = df_bin_encoded['ws_binary']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42,stratify=y)

# making agg columns for training set 
X_train['agg_shy'] = (X_train['Shyness_novel'] + X_train['Shyness_strangers']) / 2
X_train['agg_aggress_people'] = (X_train['Aggression_owner'] + X_train['Aggression_stranger']) / 2 
X_train.drop(columns=['Shyness_novel', 'Shyness_strangers', 'Aggression_stranger', 'Aggression_owner'], inplace=True)
X_train.columns

# making agg columns for testing set
X_test['agg_shy'] = (X_test['Shyness_novel'] + X_test['Shyness_strangers']) / 2
X_test['agg_aggress_people'] = (X_test['Aggression_owner'] + X_test['Aggression_stranger']) / 2
X_test.drop(columns=['Shyness_novel', 'Shyness_strangers', 'Aggression_stranger', 'Aggression_owner'], inplace=True)
X_test.columns

Index(['Age', 'Gender', 'Neuter_status', 'Weaning_age', 'Outdoors',
       'Other_cats', 'Activity_level', 'Contact_people', 'Aggression_cats',
       'Grooming', 'Behaviour_problem', 'Breed_group_ABY', 'Breed_group_BEN',
       'Breed_group_BRI', 'Breed_group_BUR', 'Breed_group_CRX',
       'Breed_group_DRX', 'Breed_group_EUR', 'Breed_group_HCS',
       'Breed_group_KOR', 'Breed_group_MCO', 'Breed_group_NFO',
       'Breed_group_ORI', 'Breed_group_PER', 'Breed_group_RAG',
       'Breed_group_RUS', 'Breed_group_SBI', 'Breed_group_SIB',
       'Breed_group_TUV', 'Breed_group_other', 'agg_shy',
       'agg_aggress_people'],
      dtype='object')

In [57]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled_encoded = scaler.transform(X_train)
X_test_scaled_encoded = scaler.transform(X_test)

rus = RandomUnderSampler()
X_resampled_rus, y_resampled_rus = rus.fit_resample(X_train_scaled_encoded, y_train)

model12 = create_mlp(64, 'adam', X_resampled_rus)
model12.fit(X_resampled_rus, y_resampled_rus, epochs=10, batch_size=32)

y_pred = model12.predict(X_test_scaled_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("old data rus adam 64", y_test, y_pred)
cm

Epoch 1/10
79/79 [==============================] - 1s 940us/step - loss: 0.6668 - accuracy: 0.6077 - recall: 0.5778
Epoch 2/10
79/79 [==============================] - 0s 921us/step - loss: 0.6353 - accuracy: 0.6481 - recall: 0.6415
Epoch 3/10
79/79 [==============================] - 0s 921us/step - loss: 0.6200 - accuracy: 0.6591 - recall: 0.6373
Epoch 4/10
79/79 [==============================] - 0s 946us/step - loss: 0.6087 - accuracy: 0.6706 - recall: 0.6760
Epoch 5/10
79/79 [==============================] - 0s 921us/step - loss: 0.6001 - accuracy: 0.6785 - recall: 0.6718
Epoch 6/10
79/79 [==============================] - 0s 908us/step - loss: 0.5901 - accuracy: 0.6821 - recall: 0.6683
Epoch 7/10
79/79 [==============================] - 0s 921us/step - loss: 0.5831 - accuracy: 0.6952 - recall: 0.6997
Epoch 8/10
79/79 [==============================] - 0s 1ms/step - loss: 0.5722 - accuracy: 0.6971 - recall: 0.6967
Epoch 9/10
79/79 [==============================] - 0s 921us/step 

C:\Users\gwent\AppData\Local\Temp\ipykernel_19588\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[623, 544],
       [184, 358]], dtype=int64)

In [58]:
# for reference, cat-pstone 1 model was 0.622, 0.519, 0.446, 0.635
# it really doesn't do any better than a weighted LR model, in either dataset
master_scores.sort_values('Recall', ascending=False)

,Model,Recall,F1,Precision,Accuracy
15,old data rus adam 64,0.661,0.496,0.397,0.574
11,"no groom, RUS, 64, adam",0.601,0.509,0.441,0.631
8,"RUS 1 hidden, 64 nodes, adam",0.599,0.498,0.426,0.615
14,log reg no grooming,0.593,0.504,0.437,0.628
12,"class_weight='balanced', LogReg",0.588,0.502,0.438,0.629
10,"RUS, adam, batch 25",0.556,0.478,0.419,0.614
9,"ROS 1 hidden, 64 nodes, adam",0.523,0.472,0.431,0.628
6,"1 hidden, 250 nodes, adam",0.298,0.368,0.482,0.675
7,"1 hidden, 300 nodes, adam",0.277,0.355,0.492,0.679
4,"1 hidden, 100 nodes, adam",0.274,0.343,0.458,0.666
